<a href="https://colab.research.google.com/github/jasonwashburn/VWGeojsonViewer/blob/main/VW_GeoJSON_Viewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install folium -U

Requirement already up-to-date: folium in /usr/local/lib/python3.7/dist-packages (0.12.1)


In [2]:
import os
import json
import folium
from folium.features import GeoJsonPopup, GeoJsonTooltip
import ipywidgets as widgets

In [3]:
product_properties = {
    'icing': {
        'color_property': 'intensity',
        'extents': ['base', 'top'],
        'color_property_values': {
            'Lgt-Rime': "#CC945C",
            'Lgt-Mixed': "#86E656",
            'Lgt-Clear': "#DD99FF",
            'Mdt-Rime': "#664524",
            'Mdt-Mixed': "#00AA00",
            'Mdt-Clear': "#FF55FF",
            'Svr-Rime': "#CC7722",
            'Svr-Mixed': "#0088AA",
            'Svr-Clear': "#FF0000",
        }
    },
    'turbulence': {
        'color_property': 'intensity',
        'extents': ['base', 'top'],
        'color_property_values': {
            'LGT': "#9C9C9C",
            'LGT-MDT': "#99CCFF",
            'MDT': "#3333FF",
            'MDT-SVR': "#FFFF00",
            'SVR': "#FF0000",
            'SVR-XTRM': "#FF77FF",
            'XTRM': "#9911FF",
            'SVR-MWV': "#FF8200" # Super obvious orange since type not implemented yet
        }
    },
    'TStorms': {
        'color_property': 'coverage',
        'extents': ['top'],
        'color_property_values': {
            'ISOLD': "#33FF00",
            'FEW': "#FFFF00",
            'SCT': "#FF6600",
            'NMRS': "#FF0000",
        }
    }   
}


In [59]:
def build_geojson_filepath_list(path="./geojson/") -> list:
    files = os.listdir(path)
    geojson_files = [path + file for file in files if ".json" in file]
    return geojson_files

def build_geojson_file_list(path="./geojson/") -> list:
    files = os.listdir(path)
    geojson_files = [file for file in files if ".json" in file]
    return geojson_files

def list_geojson_files(path="./geojson") -> None:
    for i, geojson_file in enumerate(build_geojson_filepath_list()):
        print(f"{i}: {geojson_file}")

def load_geojson_file(file) -> dict:
    with open(file, 'r') as data:
        json_data = json.loads(data.read())
    return json_data

def get_product_type(file) -> str:
    json_data = load_geojson_file(file)
    if json_data['features'][0]['properties']['feature_type']:
        return json_data['features'][0]['properties']['feature_type']

def get_product_feature_properties(file) -> str:
    json_data = load_geojson_file(file)
    if json_data['features'][0]['properties']:
        return json_data['features'][0]['properties']  

def style_function(feature) -> dict:
    feature_type = feature['properties']['feature_type']
    feature_properties = feature['properties']
    if feature_type == 'icing':
        color_lookup_value = feature_properties['intensity'] + '-' + feature_properties['type']
    elif feature_type == 'TStorms':
        color_lookup_value = feature_properties['coverage']
    elif feature_type == 'turbulence':
        color_lookup_value = feature_properties['intensity']

    color_code = product_properties[feature_type]['color_property_values'][color_lookup_value]
    return { 
        'fillColor': color_code,
        'fillOpacity': 0.6,
        'lineOpacity': 1,
        'weight': .5
    }

def build_geojson_map(geojson_file):
    geojson_data = load_geojson_file(geojson_file)
    geojson_properties = get_product_feature_properties(geojson_file)

    tooltip = GeoJsonTooltip(
        fields=[prop for prop in geojson_properties],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid blue;
            border-radius: 10px;
            box-shadow: 3px;
        """,
    )

    popup = GeoJsonPopup(
        fields=[prop for prop in geojson_properties],
        localize=True,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid blue;
            border-radius: 10px;
            box-shadow: 3px;
        """,
    )


    m = folium.Map(location=[0,0],zoom_start=2, min_zoom=1, zoom_snap=0.25, max_bounds=True, box_zoom=True, max_bounds_viscosity=0.9, width="90%", height="80%")
    folium.GeoJson(data=geojson_data, tooltip=tooltip, popup=popup, style_function=style_function).add_to(m)

    return m

def dropdown_file_selection_eventhandler(change):
    selected_file = build_geojson_filepath_list()[change.new]
    output_map.clear_output()
    with output_map:
        m = build_geojson_map(selected_file)
        display(m)

def btn_refresh_files_eventhandler(event):
    dropdown_file_selection.options = [(filenum, file) for (file, filenum) in enumerate(build_geojson_file_list())]

btn_refresh_files = widgets.Button(description="Refresh List")
btn_refresh_files.on_click(btn_refresh_files_eventhandler)
output_map = widgets.Output()
dropdown_file_selection = widgets.Dropdown(options=[(filenum, file) for (file, filenum) in enumerate(build_geojson_file_list())], description="Select File:")
dropdown_file_selection.observe(dropdown_file_selection_eventhandler, names="value")


In [51]:
display( dropdown_file_selection, btn_refresh_files)

Dropdown(description='Select File:', options=(('FITL_LLICG_2021052115Z.json', 0), ('FITL_ULICG_2021052203Z.jso…

Button(description='Refresh List', style=ButtonStyle())

In [60]:
display(output_map)

Output()